In [3]:
import pandas as pd
import ast
import math
from math import radians, cos, sin, asin, sqrt
# from numpy import cos, sin, arcsin as asin, sqrt
import numpy as np
from math import pi
import csv
from csv import writer
from csv import reader
from flask import Flask, request, jsonify
import json
import gmplot

def inidist(lat1, lat2, lon1, lon2):
    difflon = lon2 - lon1
    difflat = lat2 - lat1
    a = sin(difflat / 2)**2 + cos(lat1) * cos(lat2) * sin(difflon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    # Radius of earth
    return c*r
#c = 12742 * asin(sqrt(sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2))

def enumcircle(data):
    circles = set()
    circles2 = set()
    for i in data.index:
        for j in data.index:
            if data['row'][i] != data['row'][j]:
                circles.add((data['pickup_latitude'][i], data['pickup_longitude'][i], inidist(data['pickup_latitude'][i], data['pickup_latitude'][j], data['pickup_longitude'][i], data['pickup_longitude'][j])))
    for i in circles:
        if not np.isnan(np.min(i)):
            circles2.add(i)
    postlikelihood = {}
    for i in circles2:
        I=0
        C=0
        B=len(data.index) * (pi * i[2] * i[2]) / 608000
        #change area(denominator) according to the place
        #finding value of C
        for k in data.index:
            if i[2] > inidist(i[0],data['pickup_latitude'][k] ,i[1], data['pickup_longitude'][k]):
                C+=1
        #Updating I
        if C>=B:
            I=1
        if I == 0:
            likelihood = 0
        else:
            likelihood = math.log(((C/B)**C)*(((len(data.index)-C)/(len(data.index)-B))**(len(data.index)-C))*I)
        postlikelihood[i] = likelihood
    fn = {'Circle': [], 'Likelihood': []}
    for i, j in postlikelihood.items():
        fn['Circle'].append(i)
        fn['Likelihood'].append(j)
    pd.DataFrame(fn).to_csv('circle.csv')



def csv_radius_finder(csv_file_path):
    aa = pd.read_csv(csv_file_path)
    data = aa[['pickup_datetime', 'pickup_latitude', 'pickup_longitude']]
    data['row'] = [i for i in range(1, len(data) + 1)]
    enumcircle(data)
    return data


#driver code
#csv_file_path = input('Enter the absolute path of the CSV file: ')
csv_file_path = './time1to2.csv'
data = csv_radius_finder(csv_file_path)

gmap3 = gmplot.GoogleMapPlotter('41.8781', '-87.6298',3)
gmap3.scatter(data.iloc[:1000]['pickup_latitude'].values, data.iloc[:1000]['pickup_longitude'].values)
circles = pd.read_csv('circle.csv')
for i in range(10):
    cir = ast.literal_eval(circles.iloc[i].values[1])
    gmap3.circle(cir[0], cir[1], cir[2], face_alpha=0, ew=3, color='red')
gmap3.draw("map13.html")

C:\Users\91971\AppData\Local\Temp\ipykernel_1440\3551538610.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['row'] = [i for i in range(1, len(data) + 1)]
